In [1]:
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def readdart(fname, t1, t2, t_landfall=None):
    """
    Read DART data from file fname.
    t1, t2 = upper and lower bounds on times to return t and eta.
    t_quake = time of earthquake, a datetime object, if known.

    Returns:
    t = array of datetime objects for time series,
    eta = surface elevation at each time,
    t_sec = times in seconds since t_quake (or since t1 if t_quake==None).
    """
    columns = ['Time', 'newlevel']
    heights = []
    t = []
    t_sec = []
    data = pd.read_csv(fname, header=None, names=columns)

    for x in data.Time:
        if (x != 'Time'):
            datetime_object = datetime.datetime.strptime(x, '%x  %H:%M')
            t.append(datetime_object)

    for y in data.newlevel:
        if (y != 'newlevel'):
            heights.append(float(y))   
             
    if t_landfall is not None:
        t0 = t_landfall
    else:
        t0 = t1

    # print t0
    for m in t:
        n = m - t0
        t_sec.append(n.total_seconds())

    # # Compute seconds past t0 for each element of t:
    t = np.array(t)
    heights = np.array(heights)
    t_sec = np.array(t_sec)

    return t, t_sec, heights

In [3]:
def plot_datetime(t, heights, fig=None, axes=None):
    """
    Plot depth eta vs. time t where t is an array of datetime objects.
    fig is a matplotlib.figure.Figure object or an integer.
    ax is a matplotlib.axis object.
    Improves the formatting of tick labels to make them readable.
    """
    from matplotlib.dates import DateFormatter
    formatter = DateFormatter('%x  %H:%M')
    if fig==None:
        fig = plt.figure()
    elif type(fig) is int:
        fig = plt.figure()
    if axes==None:
        fig = plt.figure()
    axes.xaxis.set_major_formatter(formatter)
    plt.xticks(rotation=25)

    fig.autofmt_xdate()
    plt.plot(t,heights)
    return fig, axes


In [4]:
def plotdart(fname, t_landfall):
    t1 = t_landfall - datetime.timedelta(0, 36*3600)
    t2 = t_landfall + datetime.timedelta(0, 24*3600)
    t, t_sec, heights = readdart(fname, t1, t2, t_landfall)
    plot_datetime(t, heights)
    return t,t_sec,heights

In [5]:
def fit_tide_poly(t, eta, degree, t1fit, t2fit, t1out, t2out):
    """
    Fit a polynomial of the specified degree to data in the range t1fit <= t <= t2fit.
    Returns the coefficents c of c[0] + c[1]*t + ...
    and detided data eta_notide over the range t1out <= t <= t2out.
    """
    from numpy.linalg import lstsq
    
    # select subset of t, eta where fit is done:
    mask = ((t>=t1fit) & (t<=t2fit)) 
    tfit = t[mask]
    # print type(tfit)
    etafit = eta[mask]
    
    
    # select subset of t, eta for output:
    mask = ((t>=t1out) & (t<=t2out)) 
    tout = t[mask]
    etaout = eta[mask]

    
    # Scale data so matrix is well-conditioned:
    scale_factor = tfit[0]
    tfit = tfit/scale_factor
    tout = tout/scale_factor
    
    # Use Newton polynomial basis using these points:
    tpts = np.linspace(tfit.min(),tfit.max(),degree+1)
    
    # Form A matrix Afit for least squares fit and
    # Aout for applying fit to output data:
    Afit = np.ones((len(tfit),degree+1))
    Aout = np.ones((len(tout),degree+1))
    for j in range(1,degree+1):
        Afit[:,j] = Afit[:,j-1] * (tfit - tpts[j])
        Aout[:,j] = Aout[:,j-1] * (tout - tpts[j])
        
    # Performs least squares fit:
    c = lstsq(Afit,etafit)[0]
    
    #import pdb; pdb.set_trace()
    
    # evaluate polynomial at the output times:
    etaoutfit = np.dot(Aout,c)
    
    # evaluate polynomial at the fit times:
    etafit2 = np.dot(Afit,c)
    
    # Compute de-tided values by subtracting fit values from raw data:
    tout = tout*scale_factor
    tfit = tfit*scale_factor
    t_notide = tout/24/3600.0
    eta_notide =  etaout - etaoutfit
    
    # plot fit and de-tided data:
    plt.figure(70,figsize=(8,8))
    plt.clf()
    plt.subplot(211)
    plt.plot(tfit,etafit,'b')
    plt.plot(tout,etaout,'g')
    plt.plot(tfit,etafit2,'k')
    plt.plot(tout,etaoutfit,'r')
    plt.legend(['raw data over [t1fit, t2fit]', 'raw data over [t1out, t2out]', \
            'fit to data over [t1fit, t2fit]','fit over [t1out, t2out]'], \
             loc=0)
    plt.ymin = etafit.min() - 0.5*(etafit.max()-etafit.min())
    plt.ymax = etafit.max() + 0.5*(etafit.max()-etafit.min())
    plt.ylim([plt.ymin,plt.ymax])
    plt.subplot(212)
    plt.plot(t_notide, eta_notide,'k')
    plt.title('de-tided data over [t1out, t2out]')
        
    return c, t_notide, eta_notide